# automated FEAT
* **add these fields to your database for each run: MAKE SURE THE NAME ARE EXACTLY AS BELOW**
 * field_map_phase
 * field_map_mag
 * anatomical
 * dwell
   * this is 1/(0019,1028) where (0019,1028) is a dicom header for bandwidth
 * TE
   * presumably this is 2.46 for Siemens 3T scanners and 1.02 for Siemens 7T scanners
* currently, it doesn't seem like there is any good and reliable way to pull dwell time and echo spacing from the data, so this part will have to be manual
* **assumptions**
 * there is only one nifti file in the run_data_file directory in your spreadsheet
 * template. fsf is in the same directory as this notebook
 * feat_automation_utils.py is in the same directory as this notebook
 * fieldmap and structural have been skull stripped
 * fsl_prepare_fieldmap has been run

# load spreadsheet
<ol>
<li> set db equal to path to your csv file(same as from other pipeline/etc.) </li>
<li> set subject equal to subject initials</li>
<li> run the cell</li>
</ol>

In [33]:
from feat_automation_utils import *

## put your spreadsheet here
db = '/musc.repo/Data/nickdesisto/Imagery_preproc/Imagery_DB.csv'

# put subject initials here
subject = 'TN'

df = load_db(db,subject)
df

TEST = True
if TEST:
    #### TESTING VARS #####
    df['field_map_mag'] =  '/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris_TN/1_023_FieldMap_20140706/Naselaris_TN_14_07_06-10_53_40-DST-1_3_12_2_1107_5_2_34_18927_20140706_130923_019000_001_Naselaris_ImageryExperiment_024_fm2d2r_FieldMap.nii'
    df['field_map_phase'] ='/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris_TN/1_024_FieldMap_20140706/Naselaris_TN_14_07_06-10_53_40-DST-1_3_12_2_1107_5_2_34_18927_20140706_130923_019000_001_Naselaris_ImageryExperiment_024_fm2d2r_FieldMap.nii'
    df['anatomical']= '/musc.repo/Data/nickdesisto/anatomicals/1_012_t1_mpr_tra_iso_20140707/Naselaris_TN3_14_07_07-14_38_13-DST-1_3_12_2_1107_5_2_34_18927_20140707_153356_792000_001_Naselaris_ImageryExperiment_012__tfl3d1_t1_mpr_tra_iso.nii'
    df['dwell'] = .3
    df['run_path'] = '/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris_TN/'
    df['TE'] = 22.8
    df['st_file'] = '/home/nick/Desktop/slice_times.txt'

df = df[df.sessionID == 7]
df

,runID,date,subject,experiment,working_vol,brain_mask,sessionID,run_type,design matrix,frame_file,run_data_file,nvols,tesla,location,run_path,TR,matrix_x,matrix_y,n_slices,vox_x,vox_y,vox_z,run_code_path,run_code_file,design_matrix_path,frameFilePath,picPath,siemensRef,padVol,mag,phase,field_map_mag,field_map_phase,anatomical,dwell,TE,st_file
47,47,6/7/2014,TN,imagery.rf,/musc.repo/Data/nickdesisto/Imagery preprocced...,NaN,7,1pcp,design_matrix.npy,pcp_frame_list.txt,1_008_BOLD_1p6iso_MB2_iPAT2_Tr2_run1pcp_20140706,304,7T,cmrr,/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris...,2,130,130,70,1.6,1.6,1.6,/musc.repo/mri/7T.cmrr/Presentation/imagery.rf...,run_experiment.py,/musc.repo/mri/7T.cmrr/Presentation/imagery.rf...,/musc.repo/mri/7T.cmrr/Presentation/imagery.rf...,/musc.repo/mri/7T.cmrr/Presentation/imagery.rf...,0,0,/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris...,/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris...,/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris...,/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris...,/musc.repo/Data/nickdesisto/anatomicals/1_012_...,0.3,22.8,/home/nick/Desktop/slice_times.txt
48,48,6/7/2014,TN,imagery.rf,/musc.repo/Data/nickdesisto/Imagery preprocced...,NaN,7,1img,design_matrix.npy,img_frame_list.txt,1_010_BOLD_1p6iso_MB2_iPAT2_Tr2_run1img_20140706,304,7T,cmrr,/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris...,2,130,130,70,1.6,1.6,1.6,/musc.repo/mri/7T.cmrr/Presentation/imagery.rf...,run_experiment.py,/musc.repo/mri/7T.cmrr/Presentation/imagery.rf...,/musc.repo/mri/7T.cmrr/Presentation/imagery.rf...,/musc.repo/mri/7T.cmrr/Presentation/imagery.rf...,0,0,/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris...,/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris...,/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris...,/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris...,/musc.repo/Data/nickdesisto/anatomicals/1_012_...,0.3,22.8,/home/nick/Desktop/slice_times.txt
49,49,6/7/2014,TN,imagery.rf,/musc.repo/Data/nickdesisto/Imagery preprocced...,NaN,7,2pcp,design_matrix.npy,pcp_frame_list.txt,1_012_BOLD_1p6iso_MB2_iPAT2_Tr2_run2pcp_20140706,308,7T,cmrr,/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris...,2,130,130,70,1.6,1.6,1.6,/musc.repo/mri/7T.cmrr/Presentation/imagery.rf...,run_experiment.py,/musc.repo/mri/7T.cmrr/Presentation/imagery.rf...,/musc.repo/mri/7T.cmrr/Presentation/imagery.rf...,/musc.repo/mri/7T.cmrr/Presentation/imagery.rf...,0,0,/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris...,/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris...,/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris...,/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris...,/musc.repo/Data/nickdesisto/anatomicals/1_012_...,0.3,22.8,/home/nick/Desktop/slice_times.txt
50,50,6/7/2014,TN,imagery.rf,/musc.repo/Data/nickdesisto/Imagery preprocced...,NaN,7,2img,design_matrix.npy,img_frame_list.txt,1_014_BOLD_1p6iso_MB2_iPAT2_Tr2_run2img_20140706,308,7T,cmrr,/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris...,2,130,130,70,1.6,1.6,1.6,/musc.repo/mri/7T.cmrr/Presentation/imagery.rf...,run_experiment.py,/musc.repo/mri/7T.cmrr/Presentation/imagery.rf...,/musc.repo/mri/7T.cmrr/Presentation/imagery.rf...,/musc.repo/mri/7T.cmrr/Presentation/imagery.rf...,0,0,/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris...,/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris...,/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris...,/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris...,/musc.repo/Data/nickdesisto/anatomicals/1_012_...,0.3,22.8,/home/nick/Desktop/slice_times.txt
51,51,6/7/2014,TN,imagery.rf,/musc.repo/Data/nickdesisto/Imagery preprocced...,NaN,7,3pcp,design_matrix.npy,pcp_frame_list.txt,1_016_BOLD_1p6iso_MB2_iPAT2_Tr2_run3pcp_20140706,300,7T,cmrr,/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris...,2,130,130,70,1.6,1.6,1.6,/musc.repo/mri/7T.cmrr/Presentation/imagery.rf...,run_experiment.py,/musc.repo/mri/7T.cmrr/Presentation/imagery.rf...,/musc.repo/mri/7T.cmrr/Presentation/imagery.rf...,/musc.repo/mri/7T.cmrr/Presentation/imagery.rf...,0,0,/mnt/7T_Imagery/July.2014.Imagery.RF/Naselaris...,/mnt

#Choose an output directory

In [34]:
inputs = dict()
inputs['out_dir'] = '/home/nick/Desktop/fmri-preprocessing-pipeline/AutoFeat/'

#Options
* pretty much same as the gui
* self explanatory I think, True for yes, False for no.
* make sure True and False are capitalized.

In [35]:

inputs['Motion Correction'] = True

inputs['B0 Unwarping'] = True

inputs['Brain Extraction'] = True

inputs['Register to Structural'] = True

inputs['Spatial Smooth'] = 5 # Spatial smoothing FWHM (mm)

inputs['BBR DOF'] = 3 # Degrees of Freedom for registration to initial structural

inputs['Unwarp Direction'] = 'y-'

# selecting st file
inputs['st'] = 4

# here the feat run files are built, don't need to do anything

In [39]:
runs = build_runs(df,inputs)
fsfs = [build_fsf(run[0],run[1]) for run in runs]
df['fsf_files'] = save_fsf(inputs['out_dir'],fsfs)

## NEED TO ADD PRINTING THIS SHIT FOR CHECKING

# this is where feat is actually run, just run the cell and we're off
this should act equivalently to pressing go in the gui, but it will run for all the runs.

In [ ]:

run_feat_all(df)
